<a href="https://colab.research.google.com/github/drcruzm/misdatos/blob/master/Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos Faltantes

En general la mayoría de los conjuntos de datos con que nos enfrentamos contienen datos o valores faltantes. En esta notebook veremos como podemos lidiar con los valores faltantes con Pandas.

## Convenciones para datos faltantes

Existen dos estrategias generales para lidiar con valores faltantes en una tabla o *DataFrame*.

### Máscara de datos faltantes
Puede consistir en un array booleano del mismo tamaño que los datos indicando presencia o ausencia de cada valor o la apropiación de un bit en la represenación de los datos para indicar localmente el valor nulo de un valor.

### Valor centinela
Convención específica que puede ser un entero con valor -9999 u otro patrón poco común, o una convención más general como *NaN* que es valor especial especificado por la IEEE.

## Representación de valores faltantes en Pandas

Por su naturaleza Pythonica y su fuerte dependencia en Numpy, Pandas utiliza la estrategia de centinelas para los valores faltantes, en particular dos valores nulos ya existentes en el lenguaje: *NaN* y *None*.

### None en Numpy
El problema de usar *None* es que obliga a Python a usar objetos genéricos, en lugar de tipos específicos de datos (enteros, punto flotante, etc.), lo cual ralentiza su procesamiento y provoca errores en operaciones de tipo numérico.

In [1]:
import numpy as np
import pandas as pd

# Dato tipo objeto
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [2]:
# Comparación en tiempos de procesamiento
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
10 loops, best of 3: 68.5 ms per loop

dtype = int
100 loops, best of 3: 2.18 ms per loop



In [0]:
# Error en operaciones aritméticas
vals1.sum() #va a marcar error dato none !

### NaN (Not a Number) en Numpy
NaN por otro lado es un valor especial de punto flotante reconocido por todos los sistemas que usan la representación estándar de la IEEE. Sin embargo, hay que considerar que el valor de NaN afecta cualquier operación numérica que lo incluye.

In [3]:
# Tipo de dato float
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [4]:
# NaN en operaciones aritméticas
1 + np.nan

nan

In [5]:
# NaN en operaciones aritméticas
0 * np.nan

nan

In [6]:
# NaN en operaciones aritméticas
vals2.sum(), vals2.min(), vals2.max()  #lo q le haga a nan da nan

(nan, nan, nan)

In [7]:
# Funciones especiales para lidiar con NaN's
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### None y NaN en Pandas

En Pandas ``None`` y ``NaN`` se pueden usar de manera intercambiable convirtiéndose entre ellos cuando es apropiado.

In [8]:
# Conversión implícita
pd.Series([1, np.nan, 2, None]) #standariza a NaN todo !

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Si asignamos un valor de *None* a un tipo de dato entero, este lo convertirá a un valor *NaN* de punto flotante

In [9]:
x = pd.Series(range(2), dtype=int)
print(x)

x[0] = None
print(x)

0    0
1    1
dtype: int64
0    NaN
1    1.0
dtype: float64


## Operando con valores nulos

Para facilitar la convención de usar ``None`` y ``NaN`` como indicadores de valores nulos, existen varios métodos para detectar, remover y reemplazar valores nulos en Pandas.

- ``isnull()``: Genera una máscara booleana indicando posiciones de valores nulos
- ``notnull()``: Lo opuesto a ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos sin incluir valores nulos
- ``fillna()``: Devuelve una copia de los datos con los valores nulos imputados

In [11]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

### Detectando valores nulos

In [12]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [13]:
# Selección de datos no nulos
data[data.notnull()]

0        1
2    hello
dtype: object

### Eliminando valores nulos

En una serie de datos es simple.

In [14]:
# Serie de datos
data.dropna()

0        1
2    hello
dtype: object

Para los DataFrames hay más opciones, no es posible eliminar un simple dato, así que se elimina toda la fila o toda la columna

In [18]:
# Para DataFrames hay más opciones
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [16]:
# Eliminando filas
df.dropna()

,0,1,2
1,2.0,3.0,5


In [17]:
# Eliminando columnas
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Si no queremos eliminar todos los valores nulos, es posible limitar las filas o columnas eliminadas dependiendo de los parámetros ```how``` y ```tresh```.

In [19]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [20]:
# Solo elimina las filas o columnas donde todos los valores son nulos. El valo default es how='any'.
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [23]:
# Conserva solo las filas donde haya por lo menos el número thresh de valores no nulos
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Llenando valores nulos

En lugar de eliminar datos nulos, estos pueden ser completados con valores válidos. El valor puede ser una constante o el resultado de algún método de imputación o interpolación.

In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [25]:
# Con un valor constante
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [26]:
# Propagando el valor previo
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [27]:
# Retropropagando el valor siguiente
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [28]:
# Para DataFrames
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
# Llenando todos los NaN con un valor
df.fillna(0)

,0,1,2,3
0,1.0,0.0,2,0.0
1,2.0,3.0,5,0.0
2,0.0,4.0,6,0.0


In [30]:
# Diferentes valores para cada columna
df.fillna({1: 0.5, 3: -1})

,0,1,2,3
0,1.0,0.5,2,-1.0
1,2.0,3.0,5,-1.0
2,NaN,4.0,6,-1.0


In [31]:
# Llenando con la media
df.fillna(data.mean())

,0,1,2,3
0,1.0,2.0,2,2.0
1,2.0,3.0,5,2.0
2,2.0,4.0,6,2.0


In [32]:
# Por filas
print(df)
df.fillna(method='ffill', axis=0) #los datos faltantes vienen de arriba

     0    1  2   3
0  1.0  NaN  2 NaN
1  2.0  3.0  5 NaN
2  NaN  4.0  6 NaN


,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [33]:
# Por columnas
print(df)
df.fillna(method='ffill', axis=1) #faltantes vienen de la iza¿quierda

     0    1  2   3
0  1.0  NaN  2 NaN
1  2.0  3.0  5 NaN
2  NaN  4.0  6 NaN


,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Los métodos anteriores devuelven u nuevo objeto (Series o DataFrame), si lo que queremos es modificar el objeto original se usa el parámetro *inplace*.

In [34]:
df1 = df.copy()
df1

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [36]:
# Modificando el DataFrame

print(df1)
df1.fillna(-99, inplace=True)
df1

      0     1  2     3
0   1.0 -99.0  2 -99.0
1   2.0   3.0  5 -99.0
2 -99.0   4.0  6 -99.0


,0,1,2,3
0,1.0,-99.0,2,-99.0
1,2.0,3.0,5,-99.0
2,-99.0,4.0,6,-99.0
